In [1]:
using ReverseDiff
using ProgressMeter
using MLDataPattern
using Plots
gr()

Plots.GRBackend()

In [2]:
head(t::Tuple) = tuple(t[1])

function viewblocks{T <: NTuple}(data::AbstractArray, shapes::AbstractVector{T})
    starts = cumsum(vcat([1], prod.(shapes)))
    [reshape(view(data, starts[i]:(starts[i+1] - 1)), shapes[i]) for i in 1:length(shapes)]
end

viewblocks (generic function with 1 method)

In [145]:
predict = (flat_weights, shapes, x) -> begin
    weights = viewblocks(flat_weights, shapes)
    num_weights = sum(length, weights)
    biases = viewblocks(@view(flat_weights[(num_weights + 1):end]), head.(shapes))
    y = x
    J = eye(length(x))
    for i in 1:(length(shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(vec(w * y), vec(w * J))
end

loss = (w, shapes, x, y) -> sumabs2(y - predict(w, shapes, x))

function train(w, shapes, data, lr=0.1, momentum=0.8)
    last_descent = zeros(w)
    for batch in batchview(data, 1)
        dw = zeros(w)
        for (x, y) in batch
            sample_grad = ReverseDiff.gradient(w -> loss(w, shapes, x, y), w)
            dw .+= sample_grad
        end
        for i in 1:length(w)
            v = lr * dw[i] + momentum * last_descent[i]
            w[i] -= v
            last_descent[i] = v
        end
    end
    w
end

x = [1.0]
shapes = [(4, 1), (4, 4), (1, 4)]
w = rand(sum(prod, shapes) + sum(first, shapes))
# w = [0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 3.0]

train_data = [
begin
    x = randn()
    if x >= 0
        [x], [x, 1]
    else
        [x], [-x, -1]
    end
    end for i in 1:10];

@showprogress for i in 1:100
    train(w, shapes, train_data, 0.01, 0.5)
end

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(w, shapes, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markersize=1
)
plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 55.2446,-199.114 65.9602,-182.957 76.6758,-166.8 87.3915,-150.644 98.1071,-134.487 108.823,-118.331 119.538,-102.174 130.254,-86.0177 140.97,-69.8611 151.685,-53.7046 
 162.401,-37.548 173.117,-21.3914 183.832,-5.23488 194.548,10.9217 205.264,27.0782 215.979,43.2348 226.695,59.3914 237.41,75.5479 248.126,91.7045 258.842,107.861 
 269.557,125.393 280.273,142.815 290.989,154.837 301.704,166.859 312.42,178.714 323.136,179.411 333.851,173.477 344.567,158.072 355.283,142.547 365.998,127.023 
 376.714,111.498 387.429,95.9739 398.145,80.4494 408.861,64.925 419.576,49.4005 430.292,33.876 441.008,18.3515 451.723,2.82706 462.439,-12.6974 473.155,-28.2219 
 483.87,-43.7464 494.586,-59.2708 505.302,-74.7953 516.017,-90.3198 526.733,-105.844 537.448,-121.369 548.164,-136.893 558.88,-152.418 569.595,-167.942 580.311,-183.467 
 
 "/>

In [144]:
predict = (flat_weights, shapes, x) -> begin
    weights = viewblocks(flat_weights, shapes)
    num_weights = sum(length, weights)
    biases = viewblocks(@view(flat_weights[(num_weights + 1):end]), head.(shapes))
    y = x
    J = eye(length(x))
    for i in 1:(length(shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(vec(w * y), vec(w * J))
end

loss = (w, shapes, x, y) -> sumabs2(y[1] - predict(w, shapes, x)[1])

function train(w, shapes, data, lr=0.1, momentum=0.8)
    last_descent = zeros(w)
    for batch in batchview(data, 1)
        dw = zeros(w)
        for (x, y) in batch
            sample_grad = ReverseDiff.gradient(w -> loss(w, shapes, x, y), w)
            dw .+= sample_grad
        end
        for i in 1:length(w)
            v = lr * dw[i] + momentum * last_descent[i]
            w[i] -= v
            last_descent[i] = v
        end
    end
    w
end

x = [1.0]
shapes = [(4, 1), (4, 4), (1, 4)]
w = rand(sum(prod, shapes) + sum(first, shapes))
# w = [0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 3.0]

train_data = [
begin
    x = randn()
    if x >= 0
        [x], [x, 1]
    else
        [x], [-x, -1]
    end
    end for i in 1:10];

@showprogress for i in 1:100
    train(w, shapes, train_data, 0.01, 0.5)
end

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(w, shapes, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markersize=1
)
plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 55.2446,-125.536 65.9602,-115.566 76.6758,-105.595 87.3915,-95.6254 98.1071,-85.6552 108.823,-75.6851 119.538,-65.715 130.254,-55.7449 140.97,-45.7748 151.685,-35.8047 
 162.401,-25.8346 173.117,-15.8645 183.832,-5.89435 194.548,4.07576 205.264,13.4708 215.979,22.6707 226.695,31.8705 237.41,41.0704 248.126,50.2702 258.842,59.4701 
 269.557,68.6699 280.273,77.2226 290.989,77.2693 301.704,77.316 312.42,77.3626 323.136,77.4093 333.851,77.4559 344.567,77.5026 355.283,77.5493 365.998,77.5959 
 376.714,77.6426 387.429,77.6892 398.145,77.7359 408.861,77.7826 419.576,77.8292 430.292,77.8759 441.008,77.9225 451.723,77.9692 462.439,78.0159 473.155,78.0625 
 483.87,78.1092 494.586,78.1558 505.302,78.2025 516.017,78.2492 526.733,78.2958 537.448,78.3425 548.164,78.3891 558.88,78.4358 569.595,78.4825 580.311,78.5291 
 
 "/>